![alt text](images/uspas.png)
# VUV and X-ray Free Electron Lasers
# Running Genesis with lume-genesis
#### In this session, we will use lume-genesis to do the following:
- make and edit a genesis template file
- run a genesis simulation
- scan several inputs/outputs

Some comments and parameter descriptions taken from the Genesis manual: http://genesis.web.psi.ch/download.html   

##### Instructors: D. Nguyen, P. Anisimov, N. Neveu
##### Teaching Assistant: Y.S. Li
----------

# 1. Create and run the default genesis template file 

In [ ]:
!which genesis2

In [ ]:
!which genesis2-mpi

In [ ]:
!genesis2

In [ ]:
!genesis2 template.in

In [ ]:
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
plt.style.use(r'PaperDoubleFig.mplstyle')

In [ ]:
from genesis import Genesis
gen = Genesis('template.in')
gen.run()

By default, lume-genesis runs the simulation in a 'tmp' directory.   
This area is also called 'scratch' on some systems.  
Files in these types of directories are ususally volatile.   
i.e. they are not persistent after you log out/in again, or after a designated amount of time.

In [ ]:
gen.input_file

In [ ]:
# power output from the genesis template file
plt.plot(gen.output['data']['z'], gen.output['data']['power'][0]/1e9);
plt.xlabel('Undulator distance, m'); plt.ylabel('Power, GW');

# 2. Edit the file and re-run

Next we will run using mpi, and multiple cores: 

In [ ]:
genesis_bin='/home/vagrant/.local/bin/genesis2-mpi'
gen = Genesis('template.in', genesis_bin=genesis_bin)
gen.binary_prefixes = ['mpirun', '-n', '4']

Change input file parameters & run again.  
This will take about 5-6 seconds: 

In [ ]:
# undulator
gen['xlamd'] = 0.0186  # undulator wavelength, m
gen['aw0'] = gen['awd'] = 0.86  # rms undulator parameter
gen['nwig'] = int(80/gen['xlamd'])  # undulator length in xlamd

# focusing
gen['f1st'] = 5  # half F length in FODO measured in xlamd
gen['fl'] = 10   # full F length in FODO measured in xlamd
gen['quadf'] = 30  # focusing in x quadrupole gradient, T/m
gen['dl'] = 10   # full D length in FODO measured in xlamd
gen['quadd'] = 30  # defocusing in x quadrupole gradient, T/m
gen['drl'] = 100 # full O length in FODO measured in xlamd

# electron beam
gen['curpeak'] = 3000  # current, A
gen['curlen'] = 0  # negative or 0 for flattop; positive for Gaussian
gen['gamma0'] = 12e9/0.511e6  # beam energy, mc^2
gen['delgam'] = 1.5e-4*gen['gamma0']  # relative energy spread
gen['rxbeam'] = 1.2038964357474105e-05  # rms size, m
gen['rybeam'] = 1.042244688359981e-05  # rms size, m
gen['emitx'] = gen['emity'] = 0.2e-6  # normalized emittance, m rad
gen['npart'] = 2**10  # number of macroparticles in a bucket

# radiation at resonant condition
gen['xlamds'] = gen['xlamd']*(1+gen['aw0']**2)/(2*gen['gamma0']**2)
gen['prad0'] = 1e4  # shot noise power, W
gen['zrayl'] = 24  # Rayleigh length, m
gen['zwaist'] = 0  # focul point location, m

# mesh
gen['ncar'] = 151  # number of mesh points, ODD is advised
gen['dgrid'] = 100e-6  # [-dgrid, dgrid], m

# simulation
gen['delz'] = 1  # integration step measured in xlamd
gen.run()
gen.output['run_info']

In [ ]:
gen.output['data'].keys()

In [ ]:
plt.plot(gen.output['data']['z'], gen.output['data']['power'][0]/1e9);
plt.xlabel('Undulator distance, m'); plt.ylabel('Power, GW');

In [ ]:
plt.plot(gen.output['data']['z'], 1e6*gen.output['data']['xrms'][0], 'b')
plt.plot(gen.output['data']['z'], 1e6*gen.output['data']['yrms'][0], 'g')
plt.ylabel(r'$\sigma,\ \mu m$')
plt.xlabel(r'Undulator distance, m')
plt.show()

This scan should take ~30 seconds:

In [ ]:
gen['iscan'] = 12 # Scanning rms beam size RYBEAM
gen['nscan'] = 16  # default = 3
gen['svar'] = 0.025 # Defines the scan range of the selected scan parameter. 
# The parameter is varied between (1-SVAR) and (1+SVAR) of its initial value. 
# One exception is the scan over ISEED where the random number generator is not reinitialized.
gen.run()
gen.output['run_info']

In [ ]:
gen.output['data']['yrms'].shape

All scan results plotted at once: 

In [ ]:
plt.plot(gen.output['data']['z'], 1e6*gen.output['data']['yrms'].T)
plt.xlabel('Undulator distance, m')
plt.ylabel(r'$\sigma_y,\ \mu m$')
plt.show()

In [ ]:
plt.plot(1e6*gen.output['data']['current'], 1e6*np.std(gen.output['data']['yrms'], axis=1))
plt.xlabel(r'$\sigma_y(0),\ \mu m$')
plt.ylabel(r'$std[\sigma_y(z)],\ \mu m$')
plt.show()

Plotting the scan with the minimum std:

In [ ]:
plt.plot(gen.output['data']['z'], 1e6*gen.output['data']['yrms'][np.argmin(np.std(gen.output['data']['yrms'], axis=1))])
plt.ylabel(r'$\sigma_y,\ \mu m$')
plt.xlabel(r'Undulator distance, m')
plt.show()

Re-run the simulation with the min $sigma_y$:

In [ ]:
rybeam = gen.output['data']['yrms'][np.argmin(np.std(gen.output['data']['yrms'], axis=1))][0]
gen['rybeam'] = rybeam

In [ ]:
gen['iscan'] = 0
gen['nscan'] = 0
gen['svar'] = 0.025
gen.run()
gen.output['run_info']

In [ ]:
plt.plot(gen.output['data']['z'], gen.output['data']['power'][0]/1e9);
plt.xlabel('Undulator distance, m'); plt.ylabel('Power, GW');
plt.show()

In [ ]:
plt.plot(gen.output['data']['z'], 1e6*gen.output['data']['xrms'][0], 'b')
plt.plot(gen.output['data']['z'], 1e6*gen.output['data']['yrms'][0], 'g')
plt.ylabel(r'$\sigma,\ \mu m$')
plt.xlabel(r'Undulator distance, m')
plt.show()

In [ ]:
plt.plot(gen.output['data']['z'], gen.output['data']['qfld'])
plt.xlim([0, 80])
plt.xlabel("Undultor distance, m")
plt.ylabel("dB/dz, T/m")
plt.twinx()
plt.plot(gen.output['data']['z'], gen.output['data']['aw'], 'r')
plt.ylabel("AW")
plt.show()

In [ ]:
plt.plot(gen.output['data']['z'], gen.output['data']['energy'][0])

Scan will take about 88 seconds:

In [ ]:
gen['iscan'] = 4  # xlambs scan
gen['nscan'] = 52
gen['svar'] = 0.0025
gen.run()
gen.output['run_info']

In [ ]:
gen.output['data']['power'].shape

In [ ]:
plt.plot(1e10*gen.output['data']['current'], np.max(gen.output['data']['power']/1e9, axis=1), '.-');
plt.xlabel(r'$\lambda_{x-ray}$, A'); plt.ylabel('Power, GW');
plt.xlim([1e10*gen.output['data']['current'][0], 1e10*gen.output['data']['current'][-1]])
plt.title('Resonance curve')
plt.show()

In [ ]:
max_lamds=gen.output['data']['current'][np.argmax(np.max(gen.output['data']['power'], axis=1))]
print(f"Expected wavelength was {gen['xlamds']} but we have found {max_lamds}")
1-max_lamds/gen['xlamds']

In [ ]:
plt.semilogy(gen.output['data']['z'], gen.output['data']['power'].T);
plt.xlabel('Undulator distance, m'); plt.ylabel('Power, W');
plt.xlim([0, gen.output['data']['z'][-1]])
plt.show()

In [ ]:
def gain(gen, n):
    all_gain = []
    for i in range(gen['nscan']):
        power1 = gen.output['data']['power'][i][n+1]
        power0 = gen.output['data']['power'][i][n-1]
        z1 = gen.output['data']['z'][n+1]
        z0 = gen.output['data']['z'][n-1]
        gain = (power1-power0)/(z1-z0)/gen.output['data']['power'][i][n]
        all_gain.append(gain)
    return all_gain

In [ ]:
gen.output['data']['z'].shape

In [ ]:
n1 = 2090 + 50 
g1 = gain(gen, n1)
plt.plot(1e10*gen.output['data']['current'], g1, '.-', label=f"{gen.output['data']['z'][n1]} m");

n1 = 2090 
g1 = gain(gen, n1)
plt.plot(1e10*gen.output['data']['current'], g1, '.-', label=f"{gen.output['data']['z'][n1]} m");

n1 = 2090 - 50 
g1 = gain(gen, n1)
plt.plot(1e10*gen.output['data']['current'], g1, '.-', label=f"{gen.output['data']['z'][n1]} m");

n1 = 2090 - 100
g1 = gain(gen, n1)
plt.plot(1e10*gen.output['data']['current'], g1, '.-', label=f"{gen.output['data']['z'][n1]} m");

plt.title("Gain curve")
plt.xlabel(r'$\lambda_{x-ray}$, A'); plt.ylabel('Gain, $m^{-1}$');
plt.xlim([1e10*gen.output['data']['current'][0], 1e10*gen.output['data']['current'][-1]])
plt.legend()
plt.show()

Will take about 6 seconds: 

In [ ]:
gen['xlamds'] = max_lamds
gen['iscan'] = 0
gen['nscan'] = 0
gen['svar'] = 0.025
gen.run()
gen.output['run_info']

In [ ]:
plt.plot(gen.output['data']['z'], gen.output['data']['power'][0]/1e9);
plt.xlabel('Undulator distance, m'); plt.ylabel('Power, GW');
plt.show()

In [ ]:
np.max(gen.output['data']['power'][0]/1e9)